#### Overview

manipulating tables q-sql, since many of them resemble their SQL counterparts in form or function. The usual suspects such as insert, select, update, are present, as well as functionality that is not available in traditional SQL. But appearances can be deceiving: there are some significant differences in the syntax and behavior.

The first important difference is that a q table has ordered rows and columns.

A second difference is that a q table is stored physically as a collection of column lists. This means that operations on column data are vector operations. Moreover, for simple column lists, atomic, aggregate and uniform functions applied to columns are especially simple and fast since they reduce to direct memory addressing.

#### Inserting Records

Generic inseration
there are a few way
1. targeted with fields
2. naked based on position

you can insert rows into the table,t via { ,: }

In [45]:
t:([] name:`symbol$(); iq:`int$())

//You can insert with fields
t,:`name`iq!(`Beebledrox;42)
t,:`name`iq!(`Dent,92)

//you can insert without as well
t,:(`dog,123)
t,:(`cat,111)
t,:(`mouse,222)

In [46]:
t

name       iq 
--------------
Beebledrox 42 
Dent       92 
dog        123
cat        111
mouse      222


##### Key table
this is like sql where you can have a key value in the table
which means that there can only be 1 unique value for that column

in this case we have eid as the key

In [9]:
kt:([eid:`long$()] name:`symbol$(); iq:`long$())
kt,:(1002; `Dent; 98)

In [5]:
kt

eid | name iq
----| -------
1002| Dent 98


In [7]:
kt,:(1002; `Dent; 101)

In [8]:
kt

eid | name iq 
----| --------
1002| Dent 101


#### Basic Insertation

the function insert -> ,: works in the same way to insert

In [68]:
t:([] name:`Dent`Dan`dant; iq:1 2 3)
t

name iq
-------
Dent 1 
Dan  2 
dant 3 


In [69]:
`t insert (`name`iq)!(`danny;4)
`t insert (`marvin;5)

,3


,4


In [70]:
t

name   iq
---------
Dent   1 
Dan    2 
dant   3 
danny  4 
marvin 5 


In [71]:
//Truncation of table
t:3#t

In [72]:
//You are insert a table into an existing table
`t insert ([] name:`sally`jane;iq:6 9)

3 4


In [73]:
t

name  iq
--------
Dent  1 
Dan   2 
dant  3 
sally 6 
jane  9 


#### Bulk insertation of table

It is also possible to bulk insert naked field values but there is a twist. To bulk insert naked field values, you provide a list of columns not a list of rows.

In [77]:
t:([] name:`symbol$(); iq:`int$())
//t:([] name:(); iq:())

In [75]:
`t insert (`james`norman; 123 234)

0 1


In [76]:
t

name   iq 
----------
james  123
norman 234


#### Insert and Foreign Keys

When inserting data into a table that has foreign key(s), the values destined for the foreign key column(s) are checked to ensure that they appear in the primary key column(s) pointed to by the foreign key(s).

for you to insert anything into tdetails there must exist in kt already

In [1]:
kt:([eid:1001 1002 1003] name:`james`norman`jian; iq: 100 200 300)
ktdetails:([] eid:`kt$1003 1002 1001 1002 1001; sc:126 36 92 39 98)

In [2]:
`ktdetails insert( 1002; 42)

,5


In [2]:
`ktdetails insert( 1042; 11)

[0;31mcast[0m: [0;31mcast[0m

##### Inserting into key tables

same as the previous using the insert command

In [3]:
`kt insert(1004; `tim; 400)

,3


In [4]:
`kt insert(1005; `toom; 100)

,4


**Inserting into a table that already has the key will fail** 

In [4]:
`kt insert(1004;`johh;321)

[0;31minsert[0m: [0;31minsert[0m

**Upsert**

Upsert is similar to insert but better?

1. insert only allows pass by name
2. upsert supports both pass-by-name and pass-by-value

Lets give this a shot

In [7]:
([] c1:`a`b; c2:10 20) upsert(`c; 30)

c1 c2
-----
a  10
b  20
c  30


In [8]:
f:{t:([] c1:`a`b; c2:1 2); t upsert x}

In [9]:
f(`c; 30)

c1 c2
-----
a  1 
b  2 
c  30


**Ooo** 

this is interesting okay so right now f is a function, the first input to the function is given the name x.

the function takes in x and upserts it into table,t

In [10]:
kt

eid | name   iq 
----| ----------
1001| james  100
1002| norman 200
1003| jian   300
1004| tim    400
1005| toom   100


In [11]:
`kt upsert (1001; `changedd; 101)

`kt


In [12]:
kt

eid | name     iq 
----| ------------
1001| changedd 101
1002| norman   200
1003| jian     300
1004| tim      400
1005| toom     100


As mentioned previously insert will return an error when the key, is already inserted. However with upsert it is able to over-ride the exisitng row

**Learning The Select of q-SQL**

- the result of select is always a table
- select (...)(by) from table (where)

In [13]:
t:([] c1:`a`b`c; c2:1 2 3; c3: 1.1 2.2 3.3)

In [14]:
select from t

c1 c2 c3 
---------
a  1  1.1
b  2  2.2
c  3  3.3


In [17]:
select c1, c2 from t

c1 c2
-----
a  1 
b  2 
c  3 


In [19]:
select c1, rst:10*c2 from t

c1 rst
------
a  10 
b  20 
c  30 


In [21]:
select c1,c2,2*c2,c2+c3, string c3 from t

c1 c2 x c21 c3   
-----------------
a  1  2 2.1 "1.1"
b  2  4 4.2 "2.2"
c  3  6 6.3 "3.3"


In [23]:
//A virtual column i represents the offset of each record in the table 
select i,c1 from t

x c1
----
0 a 
1 b 
2 c 


In [32]:
select distinct from ([] c1:`a`b`a; c2:10 20 10)

c1 c2
-----
a  10
b  20


**select first or last n number of records using select[]**

 - select[] gives you the ability to reference particular colum 

In [33]:
select[2] from ([] c1:`a`b`c; c2:10 20 30)

c1 c2
-----
a  10
b  20


In [34]:
select[2] from ([] c1:`a`b`a; c2:10 20 30) where c1<>`a

c1 c2
-----
b  20


In [36]:
select[2] from ([] c1:`a`b`a`d`e; c2:10 20 30 40 50) where c1=`a

c1 c2
-----
a  10
a  30


In [38]:
// Starting row number, followed by number of rows
select[1 2] from ([] c1:`a`b`a`d`e; c2:10 20 30 40 50)

c1 c2
-----
b  20
a  30


In [43]:
// > operator makes it decending
// < operator make it ascending
select [>c1] from ([] c1:`d`a`c`b; c2:2 1 4 3)
select [<c1] from ([] c1:`d`a`c`b; c2:2 1 4 3)
select [<c2] from ([] c1:`d`a`c`b; c2:2 1 4 3)
select [2;>c1] from([] c1:`d`a`c`b; c2:2 1 4 3)

c1 c2
-----
d  2 
c  4 
b  3 
a  1 


c1 c2
-----
a  1 
b  3 
c  4 
d  2 


c1 c2
-----
a  1 
d  2 
b  3 
c  4 


c1 c2
-----
d  2 
c  4 


**Nested Columns**

Nested columns means that particular column may not be flat and has like an array instead. Generally, how you select on a column like that is the use of a lot of iterators

In [52]:
show tnest:([] c1:`a`b`c; c2:(10 20 30;40;50 60))

c1 c2      
-----------
a  10 20 30
b  40      
c  50 60   


In [56]:
// This returns an error as the field in c2 is not uniform
select avg c2 from tnest

[0;31mlength[0m: [0;31mlength[0m

In [58]:
// However we are able to apply the avg function on each field of c2
select avg each c2 from tnest

c2
--
20
40
55


In [64]:
update c3:(1.1 2.2 3.3;4.4;5.5 6.6) from tnest

c1 c2       c3         
-----------------------
a  10 20 30 1.1 2.2 3.3
b  40       4.4        
c  50 60    5.5 6.6    


In [89]:
select c1, wtavg:c2 wavg' c3 from tnest
select c1, wtavg: wavg'[c2;c3] from tnest

c1 wtavg   
-----------
a  2.566667
b  4.4     
c  6.1     


c1 wtavg   
-----------
a  2.566667
b  4.4     
c  6.1     


**Filtering with where**

same as sql, will be going through some example


In [103]:
show t:([] c1:`a`b`c; c2:10 20 100; c3:2.2 2.2 3.3)

c1 c2  c3 
----------
a  10  2.2
b  20  2.2
c  100 3.3


In [104]:
select vwap:wavg[c2;c3] from t

vwap    
--------
3.046154


In [107]:
t where t[`c2]>15

c1 c2  c3 
----------
b  20  2.2
c  100 3.3


In [108]:
tbig:100#t

In [112]:
select from tbig where i within 50 55

c1 c2  c3 
----------
c  100 3.3
a  10  2.2
b  20  2.2
c  100 3.3
a  10  2.2
b  20  2.2


In [115]:
s:50
e:55
select from tbig where i within (s;e)

c1 c2  c3 
----------
c  100 3.3
a  10  2.2
b  20  2.2
c  100 3.3
a  10  2.2
b  20  2.2


In [118]:
r1:select from t where c2>15,c3<3.0
r2:select from t where (c2>15)&(c3<3.0)
r1~r2

1b


In [3]:
t:([] c1:00:00:00.000+til 1000000;c2:1000000?`a`b;c3:1000000?100.)

In [4]:
select[5] from t

c1           c2 c3      
------------------------
00:00:00.000 a  64.78666
00:00:00.001 a  11.97604
00:00:00.002 b  43.93202
00:00:00.003 b  13.32134
00:00:00.004 a  85.98006


In [11]:
select from t where c2=`a, c1 within 00:00:00:015 00:00:00:0300

c1           c2 c3      
------------------------
00:00:00.017 a  80.83228
00:00:00.020 a  69.0548 
00:00:00.022 a  90.85562
00:00:00.023 a  76.32188
00:00:00.024 a  77.49987
00:00:00.025 a  17.98912
00:00:00.026 a  63.77097
00:00:00.027 a  5.948771
00:00:00.028 a  23.00063


**Nested columns in  where**

nest columns requires iterator, inorder to do where you are also require to use where

In [17]:
t:([] f:1.1 2.2 3.3; s:("abc";enlist "d"; "ef"))
select from t where s~\:"ef"
select from t where s like "abc"

f   s   
--------
3.3 "ef"


f   s    
---------
1.1 "abc"


In [22]:
show t:([]sym:`IBM`IBM`MSFT`IBM`MSFT;
    ex:`N`O`N`N`N;
    time:12:10:00 12:30:00 12:45:00 12:50:00 13:30:00;
    price:82.1 81.95 23.45 82.05 23.40)
select from t where price=(max;price) fby ([]sym;ex)

sym  ex time     price
----------------------
IBM  N  12:10:00 82.1 
IBM  O  12:30:00 81.95
MSFT N  12:45:00 23.45
IBM  N  12:50:00 82.05
MSFT N  13:30:00 23.4 


sym  ex time     price
----------------------
IBM  N  12:10:00 82.1 
IBM  O  12:30:00 81.95
MSFT N  12:45:00 23.45


In [27]:
t:([] c1:`a`b`a`b`c; c2:10 20 30 40 50)
t
t[`c2] group t[`c1]

c1 c2
-----
a  10
b  20
a  30
b  40
c  50


a| 10 30
b| 20 40
c| ,50


In [28]:
select sum c2 by c1 from t

c1| c2
--| --
a | 40
b | 60
c | 50


In [32]:
t:([] c1:00:00:00.000+til 1000000;c2:1000000?`a`b;c3:1000000?100.)
select avg c3 by 100 xbar c1,c2 from t

c1           c2| c3      
---------------| --------
00:00:00.000 a | 48.14446
00:00:00.000 b | 53.86302
00:00:00.100 a | 46.76909
00:00:00.100 b | 51.96799
00:00:00.200 a | 48.611  
00:00:00.200 b | 47.86834
00:00:00.300 a | 43.07052
00:00:00.300 b | 47.28464
00:00:00.400 a | 47.15773
00:00:00.400 b | 46.44672
00:00:00.500 a | 52.46684
00:00:00.500 b | 47.04083
00:00:00.600 a | 52.11636
00:00:00.600 b | 47.59234
00:00:00.700 a | 51.94419
00:00:00.700 b | 52.30139
00:00:00.800 a | 46.75963
00:00:00.800 b | 46.61795
00:00:00.900 a | 49.07632
00:00:00.900 b | 51.62584
..
